### FILES PRE-PROCESSING FOR GLOVE EMBEDDING 

In [6]:
import numpy as np
import scipy
import matplotlib
import sklearn
import torch


def load_GLOVE(PATH):

    Dictionary = {}
    weights = []

    count=0
    with open(PATH, 'r', encoding="utf-8") as f:
        
            content = f.readlines()
            for row in content:

                vector = []
                
                
                s = row.split(' ')
                Dictionary[s[0]] = count
                count +=1
                values = s[1:]
                for v in values:
                    v = v.rstrip()
                    #print(type(float(v)))

                    vector.append(float(v))
                    #print(type(vector[0]))




                

                weights.append(vector)
                
                
                

            #count+=1
            #Dictionary[line]

    return torch.tensor(weights),Dictionary
    #print(weight_tensor.size())



path="data/glove.6B.200d.txt"
weight_tensor,Dictionary = load_GLOVE(path)





print('Dictionary and Weight size',(weight_tensor.size()))
#Save Dictionary and Glove pre-trained weights
np.save('Dict_Glove200.npy', Dictionary)
np.save('Weight_Glove200.npy', weight_tensor.numpy())


Dictionary and Weight size torch.Size([400002, 50])


### EMEBEDDING CLASS same for GLOVE and Word2vec and test on it and it's 4 modalities

**batch** that can be used to spefify if is expecting only a tokenized sentence or a batch ot tokenized senteces

**by_id** to specify if the model should exspect list of tokens or already indexed senteces. 

ie. for bath = False and [0 1 4 58 15 98] with by_id=True and ['the','cat','is','on','the','table']

In [44]:
class GloveEmbedding():

    def __init__(self, weights, dict, batch, by_id ) -> None:

        self.weights = weights
        self.dict = dict
        self.batch = batch
        self.weights_leng = len(self.weights[0,:])
        self.by_id = by_id
    

    def get_weight(self,token):

        if(self.by_id == False):
            index_token = self.dict[token]
        else:
            return self.weights[token,:]

        
    
        return self.weights[index_token,:]


    def embedd(self,list_words):

        list=[]
        
        if(self.batch == False):

            for word in list_words:
                list.append(self.get_weight(word).tolist())
            

            

            return torch.tensor(list)
        
        elif(self.batch == True):

            max_pad = longest(list_words)
            #print('max padd',max_pad)

            

            for sentence in list_words:

                current_lenght = len(sentence)
                #print('current :',current_lenght)
                sencence=[]
                for word in sentence:
                    sencence.append(self.get_weight(word).tolist())
                
                

                if(max_pad>current_lenght):
                    #print('padding')
                    listofzeros = [0] * self.weights_leng

                    while max_pad>current_lenght:
                        #print('padding now')
                        sencence.append(listofzeros)
                        #print('new leng',len(sencence))
                        current_lenght = len(sencence)
                

                #print('append list',len(sencence))
                list.append(sencence)


            return torch.tensor(list)





            
def longest(list1):
    longest_list = max(len(elem) for elem in list1)
    return longest_list







#TESTING GLOVE EMBEDDING

lis=[['table','on','book','you','?'],['the','my','cat'],['table','on','book','you']]
#CHECK on longest fucntion,  used for padding in batch mode
print(longest(lis))

Ge = GloveEmbedding(weight_tensor,Dictionary,False,False)
#Check on get_weight function
print(Ge.get_weight('robin').size())
#Using BATCH = False
print(Ge.embedd(['the','my','cat']).size())



#Using BATCH = TRUE but passing no index but directly words
Ge = GloveEmbedding(weight_tensor,Dictionary,True,False)
print(Ge.embedd([['the','my','cat'],['table','on','book','you'],['table','on','book','you','?']]).size())

#Using BATCH = TRUE but with passing idexes of words
Ge = GloveEmbedding(weight_tensor,Dictionary,True,True)
print(Ge.embedd([[1,0,3,5],[0,2],[1,4,8,9,2,5]]).size())



     




5
torch.Size([100])
torch.Size([3, 100])
torch.Size([3, 5, 100])
torch.Size([3, 6, 100])


### FILES PRE-PROCESSING FOR WORD2VEC EMBEDDING 

In [6]:
import gensim
import torch
import numpy as np

def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros



def load_W2W(PATH):

    Dictionary = {}
    weights = []
    model = gensim.models.KeyedVectors.load_word2vec_format(PATH, binary=True)
    print('Model load_word2vec_format Created')
    count=0

    for index, word in enumerate(model.index_to_key):

        Dictionary[word] = index
        weights.append(model[word])
        count+=1


    #ADD PAD AND UNK 
    pad = '<PAD>'
    pad_w = zerolistmaker(300)
    un = '<UNK>'
    un_w = zerolistmaker(300)

    Dictionary[pad] = count
    weights.append(pad_w)

    Dictionary[un] = count+1
    weights.append(un_w)
        

    return torch.tensor(weights),Dictionary






path="GoogleNews-vectors-negative300.bin"
weight_tensor,Dictionary = load_W2W(path)


#Save Dictionary and Word2Vec pre-trained weights
print('Dictionary and Weight size :',weight_tensor.size())
np.save('Dict_W2W.npy', Dictionary)
np.save('Weight_W2W.npy', weight_tensor.numpy())


Model load_word2vec_format Created
